In [1]:
CONFIG_PATH = "../config.yaml"
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation
dataloader, num_nodes, x_df, y_df = data_preparation(CONFIG_PATH)

The ERA5 raster has spatial dimensions: (0.10000000419197502, -0.1000000004915847)
NDVI dataset resolution: (0.043782177927351104, 0.04378217792735114)
Precipitation dataset resolution (0.10000000419197502, -0.1000000004915847)
The features have dimensions: (730, 92, 1)
The instance have dimensions: (730, 92, 1)
x shape:  (707, 12, 92, 1) , y shape:  (707, 12, 92, 1)
train x:  (511, 12, 92, 1) y: (511, 12, 92, 1)
val x:  (73, 12, 92, 1) y: (73, 12, 92, 1)
test x:  (146, 12, 92, 1) y: (146, 12, 92, 1)


In [13]:
x_df = x_df.unstack(["lat","lon"])

In [14]:
import numpy as np
time_ind = (x_df.index.values - x_df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))

In [24]:
import os 
from p_drought_indices.functions.function_clns import load_config
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import asym_adj
import pickle

CONFIG_PATH = "../config.yaml"
config = load_config(CONFIG_PATH)
adj_type = 'doubletransition'

#adj_path = os.path.join(config["DEFAULT"]["data"], "graph_net/adj_dist.pkl")
#sensor_ids, sensor_id_to_ind, adj_mx = load_adj(adj_path,  adj_type)

In [42]:
import pandas as pd
import os
resample_time="5min"
output_column_name="bikes_available"
status_df_filename = config["DEFAULT"]["data"] + "\status.csv\status.csv"

#def generate_train_val_test(status_df_filename, resample_time="5min", output_column_name="bikes_available"):
df = pd.read_csv(status_df_filename)
df.time = pd.to_datetime(df.time).dt.round('min')
print("Pivoting")
df_m = df.pivot_table(index='time', columns='station_id', values=output_column_name, aggfunc=np.min)
print("Resampling")
df_mr = df_m.resample(resample_time).mean()
#null treatment
null_quantile = df_mr.isnull().sum().quantile(0.85)
threshold_null = len(df_mr.index) - null_quantile
print('Threshold of null rows per column', null_quantile)
print('Columns to be removed', (df_mr.isnull().sum() > null_quantile).sum())
df_mrn = df_mr.dropna(thresh=threshold_null, axis='columns').interpolate()
print('Null values remaining', df_mrn.isnull().sum().sum())
# 0 is the latest observed sample.
x_offsets = np.sort(
    np.concatenate((np.arange(-11, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 13, 1))
# x: (num_samples, input_length, num_nodes, input_dim)
# y: (num_samples, output_length, num_nodes, output_dim)

Pivoting
Resampling
Threshold of null rows per column 1086.2499999999998
Columns to be removed 11
Null values remaining 0


In [79]:
df = df_mrn.copy()

add_time_in_day = True
add_day_in_week = False

num_samples, num_nodes = df.shape
data = np.expand_dims(df.values, axis=-1)
data_list = [data]
if add_time_in_day:
    time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
    time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
    data_list.append(time_in_day)
if add_day_in_week:
    day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
    day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1
    data_list.append(day_in_week)
data = np.concatenate(data_list, axis=-1)
# epoch_len = num_samples + min(x_offsets) - max(y_offsets)
x, y = [], []
# t is the index of the last observation.
min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = data[t + x_offsets, ...]
    y_t = data[t + y_offsets, ...]
    x.append(x_t)
    y.append(y_t)
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)

print("x shape: ", x.shape, ", y shape: ", y.shape)

output_dir = os.path.join(config["DEFAULT"]["data"],  "graph_net")

from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import load_dataset

num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
x_test, y_test = x[-num_test:], y[-num_test:]
for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join(output_dir, f"{cat}.npz"),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )
batch_size = 64
dataloader = load_dataset(output_dir, batch_size, batch_size, batch_size)


x shape:  (210936, 12, 59, 2) , y shape:  (210936, 12, 59, 2)
train x:  (147671, 12, 59, 2) y: (147671, 12, 59, 2)
val x:  (21096, 12, 59, 2) y: (21096, 12, 59, 2)
test x:  (42192, 12, 59, 2) y: (42192, 12, 59, 2)
